In [ ]:
# Phần code từ model.py (hàm time_dependent_integral)
from scipy.linalg import expm
import numpy as np
from qiskit.quantum_info import SparsePauliOp # Đảm bảo import

def time_dependent_integral(h_func_t, total_time, num_steps=100): # Đổi h_opt thành h_func_t để rõ ràng hơn
    """
    Tính tích phân của Hamiltonian phụ thuộc thời gian h_func_t(t) từ 0 đến total_time.
    h_func_t là một hàm nhận vào thời gian t và trả về một SparsePauliOp.
    """
    time_points = np.linspace(0, total_time, num_steps)
    # Lấy ma trận của H(0) để biết kích thước
    # Giả định rằng số qubit không đổi theo thời gian
    H_matrix_example = h_func_t(0).to_matrix() # Lấy ma trận từ SparsePauliOp
    integral_matrix = np.zeros_like(H_matrix_example, dtype=complex)

    for i in range(len(time_points) - 1):
        t_i = time_points[i]
        t_i_plus_1 = time_points[i+1]
        dt = t_i_plus_1 - t_i

        H_t_i = h_func_t(t_i).to_matrix() # Chuyển SparsePauliOp sang ma trận
        H_t_i_plus_1 = h_func_t(t_i_plus_1).to_matrix() # Chuyển SparsePauliOp sang ma trận

        # Kiểm tra tính Hermite (tùy chọn, nhưng tốt để có)
        if not np.allclose(H_t_i, np.conj(H_t_i).T):
            raise ValueError(f"Hamiltonian tại t={t_i} không Hermite.")
        if not np.allclose(H_t_i_plus_1, np.conj(H_t_i_plus_1).T):
            raise ValueError(f"Hamiltonian tại t={t_i_plus_1} không Hermite.")

        integral_matrix += (H_t_i + H_t_i_plus_1) / 2 * dt

    return integral_matrix

# --- Bây giờ kết hợp ---
from functools import partial

# Tạo một hàm h_opt_for_integral chỉ nhận vào 't'
# bằng cách cố định các tham số khác của get_hamiltonian_t_from_paper
h_opt_for_integral = partial(get_hamiltonian_t_from_paper,
                             H_static_qubit_op=H_static_qubit,
                             dipole_qubit_op=dipole_op_qubit,
                             E0=E0_paper,
                             Gamma=Gamma_paper)

# Thiết lập thời gian tổng để tính tích phân
T_total_integration = 5.0 # Ví dụ: 5 đơn vị thời gian
num_integration_steps = 200 # Số bước để tính tích phân

# Tính toán ma trận tích phân
# Trước khi gọi, đảm bảo H_static_qubit và dipole_op_qubit đã được khởi tạo đúng cách
if 'H_static_qubit' in globals() and 'dipole_op_qubit' in globals():
    try:
        integral_H_dt_matrix = time_dependent_integral(h_opt_for_integral, T_total_integration, num_steps=num_integration_steps)
        print(f"\nMa trận tích phân của H(t)dt từ 0 đến {T_total_integration}:\n", integral_H_dt_matrix)

        # Nếu bạn muốn tính toán toán tử tiến hóa thời gian U = expm(-1j * integral_H_dt_matrix)
        U_matrix = expm(-1j * integral_H_dt_matrix)
        print("\nMa trận toán tử tiến hóa thời gian U:\n", U_matrix)

        # Kiểm tra tính Unita của U
        identity_matrix = np.eye(U_matrix.shape[0])
        if np.allclose(U_matrix @ U_matrix.conj().T, identity_matrix) and \
           np.allclose(U_matrix.conj().T @ U_matrix, identity_matrix):
            print("\nMa trận U là Unita.")
        else:
            print("\nCẢNH BÁO: Ma trận U không Unita.")
            # print("U @ U_dagger:\n", U_matrix @ U_matrix.conj().T)

    except ValueError as e:
        print(f"Lỗi trong quá trình tính toán: {e}")
    except NameError as e:
        print(f"Lỗi biến chưa được định nghĩa: {e}. Đảm bảo H_static_qubit và dipole_op_qubit đã được tạo.")
else:
    print("Vui lòng chạy phần code từ biklmjroinha.py để khởi tạo H_static_qubit và dipole_op_qubit trước.")

In [ ]:
# Import các phần cần thiết từ biklmjroinha.py và các thư viện khác
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp
import numpy as np
from functools import partial # Để dễ dàng truyền tham số

# --- Phần thiết lập H_static_qubit và dipole_op_qubit (từ biklmjroinha.py) ---
# Bạn cần đảm bảo phần này được chạy để các biến này được khởi tạo
# (Đây là ví dụ, bạn cần code đầy đủ của mình ở đây)
# driver = PySCFDriver(...)
# problem = driver.run()
# hamiltonian_static_obj = problem.hamiltonian
# second_q_op_static = hamiltonian_static_obj.second_q_op()
# mapper = JordanWignerMapper()
# H_static_qubit = mapper.map(second_q_op_static) # SparsePauliOp
#
# dipole_property = problem.properties.electronic_dipole_moment
# dipole_ops_second_q = dipole_property.second_q_ops()
# dipole_op_z_second_q = dipole_ops_second_q["ZDipole"] # Hoặc tên key phù hợp
# dipole_op_qubit = mapper.map(dipole_op_z_second_q) # SparsePauliOp
#
# Gamma_paper = 0.25
# E0_paper = 0.01
# --- Kết thúc phần thiết lập ---

def H_t_from_paper_generator(t, H_static_op, dipole_op, E0, Gamma):
    """
    Tạo Hamiltonian phụ thuộc thời gian H(t) = H_static + f(t) * Dipole.
    Trả về một SparsePauliOp.
    """
    f_t = (E0 / np.pi) * Gamma / (Gamma**2 + t**2)
    time_dependent_part = dipole_op * f_t
    H_total_t = H_static_op + time_dependent_part
    return H_total_t

# Tạo một phiên bản của hàm H(t) với các tham số H_static, dipole, E0, Gamma đã được cố định
# Đảm bảo H_static_qubit và dipole_op_qubit đã được định nghĩa trước khi chạy dòng này
# Ví dụ, nếu bạn chạy trong cùng một script:
try:
    # Giả sử H_static_qubit, dipole_op_qubit, E0_paper, Gamma_paper đã được định nghĩa
    # từ phần code của biklmjroinha.py
    hamiltonian_function_from_paper = partial(H_t_from_paper_generator,
                                             H_static_op=H_static_qubit,
                                             dipole_op=dipole_op_qubit,
                                             E0=E0_paper,
                                             Gamma=Gamma_paper)
except NameError:
    print("LƯU Ý: H_static_qubit hoặc dipole_op_qubit chưa được định nghĩa.")
    print("Hãy đảm bảo bạn đã chạy phần khởi tạo Hamiltonian từ biklmjroinha.py trước.")
    hamiltonian_function_from_paper = None # Để code không bị lỗi ngay lập tức

In [ ]:
# (Trong run_vqa.py)
import sys, qiskit
sys.path.insert(0, '../../..')
import matplotlib.pyplot as plt
import numpy as np
import qoop
from qoop.compilation.qsp import QuantumStatePreparation
from qoop.core import ansatz, state, random_circuit
from qoop.backend import constant, utilities
from qoop.evolution import crossover, mutate, selection, threshold
from qoop.evolution.environment import EEnvironment, EEnvironmentMetadata
import pickle
import model
import os
import sys, qiskit
# ... các import khác ...
import model # Đảm bảo model.py có hàm time_dependent_qc mong đợi hàm h_time
import os

# --- BỎ ĐI ĐỊNH NGHĨA coefs VÀ h_time CŨ ---
# def coefs(mod):
#     ...
# mod = 'mod5'
# def h_time(t):
#     N, J, u, h, T = coefs(mod)
#     return model.XYZ_model(N,J,u,h,T,t)
# --- KẾT THÚC PHẦN BỎ ĐI ---

# THAY THẾ BẰNG HÀM SỬ DỤNG HAMILTONIAN TỪ BÀI BÁO
# Hàm này sẽ được truyền vào model.time_dependent_qc
# Đảm bảo hamiltonian_function_from_paper đã được định nghĩa và hoạt động
# (ví dụ, bằng cách import hoặc định nghĩa nó ở trên trong cùng file,
# hoặc đảm bảo nó có thể truy cập được từ một module khác)

if hamiltonian_function_from_paper is None:
    raise ValueError("`hamiltonian_function_from_paper` chưa được khởi tạo đúng cách. "
                     "Kiểm tra lại phần thiết lập Hamiltonian từ bài báo.")

# Hàm h_time mới sẽ là chính hamiltonian_function_from_paper
# hoặc một wrapper nếu cần điều chỉnh đối số/kiểu trả về
# Trong trường hợp này, model.time_dependent_qc(N, h_time, time)
# mong h_time là một hàm chỉ nhận 'time' (hoặc 't').
# hamiltonian_function_from_paper đã là một hàm chỉ nhận 't' nhờ partial.
h_time_paper = hamiltonian_function_from_paper


def compilation_fitness(qc: qiskit.QuantumCircuit):
    p0s = []
    # N, J, u, h, T = coefs(mod) # Không cần nữa nếu h_time_paper không phụ thuộc chúng
    # Nếu model.time_dependent_qc cần số qubit N, bạn cần lấy nó từ Hamiltonian
    # Ví dụ:
    num_qubits_from_ham = H_static_qubit.num_qubits if 'H_static_qubit' in globals() else 2 # Cung cấp giá trị mặc định

    times = np.linspace(0, 10, 100) # Giữ nguyên hoặc điều chỉnh nếu cần
    for time_step in times: # Đổi 'time' thành 'time_step' để tránh xung đột tên
        # Truyền h_time_paper vào đây
        # model.time_dependent_qc(N, h_func, t)
        target_U_dagger = model.time_dependent_qc(
            num_qubits_from_ham, # Số qubit
            h_time_paper,        # Hàm H(t) của bạn
            time_step            # Thời điểm hiện tại
        ).inverse()

        qsp = QuantumStatePreparation(
            u=qc,
            target_state=target_U_dagger
            ).fit(num_steps=300, metrics_func=['loss_basic'])
        p0s.append(1-qsp.compiler.metrics['loss_basic'][-1])

        # Lưu trữ kết quả (điều chỉnh tên thư mục nếu cần)
        # time_folder = os.path.join(f"times_paper_H/time_idx_{int(time_step*10)}") # Ví dụ tên thư mục mới
        # os.makedirs(time_folder, exist_ok=True)
        # qsp.save(os.path.join(time_folder, "qsp_data")) # Ví dụ cách lưu
        print(f"Hoàn thành cho time_step = {time_step:.2f}, p0 = {p0s[-1]:.4f}")


    return np.mean(p0s)

def super_evol(_depth, _num_circuit, _num_generation):
    # Lấy số qubit từ Hamiltonian (nếu đã định nghĩa)
    current_num_qubits = H_static_qubit.num_qubits if 'H_static_qubit' in globals() else 2

    env_metadata = EEnvironmentMetadata(
        num_qubits = current_num_qubits, # Sử dụng số qubit đúng
        depth = _depth,
        num_circuit = _num_circuit,
        num_generation = _num_generation,
        prob_mutate=3/(_depth * _num_circuit)
    )
    env = EEnvironment(
        metadata = env_metadata,
        fitness_func= compilation_fitness,
        selection_func=selection.elitist_selection,
        crossover_func=crossover.onepoint_crossover,
        mutate_func=mutate.layerflip_mutate,
        threshold_func=threshold.compilation_threshold
    )
    # Điều chỉnh tên file lưu trữ
    # env.set_filename(f'paper_H_d={_depth}_n_circuit={_num_circuit}_n_gen={_num_generation}')
    env.evol()
    print('done')

# main
if __name__ == '__main__':
    # num_qubits = 2 # Sẽ được xác định từ Hamiltonian
    # Đảm bảo phần khởi tạo H_static_qubit, dipole_op_qubit, E0_paper, Gamma_paper
    # và hamiltonian_function_from_paper được chạy trước khi gọi super_evol
    if hamiltonian_function_from_paper is not None and 'H_static_qubit' in globals():
        print(f"Chạy super_evol với Hamiltonian từ bài báo ({H_static_qubit.num_qubits} qubits)...")
        super_evol(4, 8, 16) # Các tham số ví dụ
    else:
        print("Không thể chạy super_evol do Hamiltonian từ bài báo chưa được thiết lập đúng cách.")